In [ ]:
!nvidia-smi

In [ ]:
#%pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12
#%pip install cucim-cu12 cupy-cuda12x

In [ ]:
#%load_ext cudf.pandas

#from cucim.skimage.exposure import rescale_intensity
import tensorflow as tf
#import cupy as cp
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split

print("Is torch using cuda? ",torch.cuda.is_available())
print("Is tensorflow using cuda? ",tf.test.is_built_with_cuda())
print("Is pandas using cuda? ",pd)


In [ ]:
name_mapping = [
    "box",
    "circularTorus",
    "cone",
    "coneOffset",
    "cylinder",
    "cylinderSlope",
    "dish",
    "mesh",
    "pyramid",
    "rectangularTorus",
    "sphere"
]

In [ ]:
def sort_by_number(texts:list[str]):
    def key(text:str):
        text = re.sub(r'.*photos_', '', text)
        text = re.sub(r'\.csv', '', text)
        text = re.sub(r'\D', '', text)
        return int(text)
    return sorted(texts, key=key)

In [ ]:
base_path = '/home/workspace/geometry-classifier/data/'

csv_files = sort_by_number(glob.glob(base_path+'photos_v3/*.csv'))

batch_size = 256

def load_and_preprocess_data(file_path, batch_size=32):
    # Load your dataset
    df = pd.read_csv(file_path)
    X = df.drop(['name', 'id'], axis=1)
    Y = df['name']
    
    # Convert to TensorFlow Dataset
    dataset = tf.data.Dataset.from_tensor_slices()
    dataset = dataset.batch(batch_size)

    return dataset


In [ ]:
csv_files